<a href="https://colab.research.google.com/github/Haavi97/ITS8040-NLSP/blob/master/Default_independent_project_guide_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!ls

git.ipynb  sample_data


The "default" project topic is about Aspect-Based Sentiment Analysis (ABSA). By "default" it is meant that you are supposed to choose this topic if you don't have any preferences about the topic yourself. You are free to choose any other sufficiently challenging topic, but please consult with the lecturer.

The task comes from the SemEval 2016 Shared Task 5: http://alt.qcri.org/semeval2016/task5/

There are different subtasks and slots in this task. You can limit you work on Task 1 Slot 1: Aspect Category Detection, and you can further focus on the restaurant domain only.

The task is: given a sentence taken from a restaurant review, your system has to decide about which aspect category of the restaurant this sentence is, if any. The number of different aspect categories is fixed. One sentnce can correspond to zero, one or more aspect categories.

Some examples:

*   "I was very disappointed with this restaurant." -> RESTAURANT#GENERAL
*   "I’ve asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away." -> SERVICE#GENERAL
*   "Chow fun was dry; pork shu mai was more than usually greasy and had to share a table with loud and rude family." -> FOOD#QUALITY, AMBIENCE#GENERAL

The Slot 3 of the task would be finding the polarity (negative, neutral, positive) of the aspect, but you don't have to implement this.

Since the data and evaluation tools for this Shared task come in quite complex form, I have implemented very basic data loading for you, together with a simplest possible sklearn-based implementation for this task.





I have packed the data for the restaurant domain for this subtask on my site. Let's download it:

In [ ]:
! wget --no-check-certificate https://www.phon.ioc.ee/~tanela/tmp/absa-en-restaurant.zip

--2021-04-28 13:07:47--  https://www.phon.ioc.ee/~tanela/tmp/absa-en-restaurant.zip
Resolving www.phon.ioc.ee (www.phon.ioc.ee)... 193.40.251.126
Connecting to www.phon.ioc.ee (www.phon.ioc.ee)|193.40.251.126|:443... connected.
  Issued certificate has expired.
	requested host name ‘www.phon.ioc.ee’.
HTTP request sent, awaiting response... 200 OK
Length: 136921 (134K) [application/zip]
Saving to: ‘absa-en-restaurant.zip’

absa-en-restaurant. 100%[===================>] 133.71K   209KB/s    in 0.6s    

2021-04-28 13:07:48 (209 KB/s) - ‘absa-en-restaurant.zip’ saved [136921/136921]



In [ ]:
! unzip absa-en-restaurant.zip

Archive:  absa-en-restaurant.zip
  inflating: ABSA16_Restaurants_Train_SB1_v2.xml  
  inflating: EN_REST_SB1_TEST.xml.gold  


There are two files: training and test files. Both are in XML formats. Let's  take a peek:

In [ ]:
! head -20 ABSA16_Restaurants_Train_SB1_v2.xml  

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Reviews>
    <Review rid="1004293">
        <sentences>
            <sentence id="1004293:0">
                <text>Judging from previous posts this used to be a good place, but not any longer.</text>
                <Opinions>
                    <Opinion target="place" category="RESTAURANT#GENERAL" polarity="negative" from="51" to="56"/>
                </Opinions>
            </sentence>
            <sentence id="1004293:1">
                <text>We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.</text>
                <Opinions>
                    <Opinion target="staff" category="SERVICE#GENERAL" polarity="negative" from="75" to="80"/>
                </Opinions>
            </sentence>
            <sentence id="1004293:2">
                <text>They never brought us complimentary noodles, ignored repeated requests for sugar, and thre

In [ ]:
! head -20 ABSA16_Restaurants_Train_SB1_v2.xml  

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Reviews>
    <Review rid="1004293">
        <sentences>
            <sentence id="1004293:0">
                <text>Judging from previous posts this used to be a good place, but not any longer.</text>
                <Opinions>
                    <Opinion target="place" category="RESTAURANT#GENERAL" polarity="negative" from="51" to="56"/>
                </Opinions>
            </sentence>
            <sentence id="1004293:1">
                <text>We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.</text>
                <Opinions>
                    <Opinion target="staff" category="SERVICE#GENERAL" polarity="negative" from="75" to="80"/>
                </Opinions>
            </sentence>
            <sentence id="1004293:2">
                <text>They never brought us complimentary noodles, ignored repeated requests for sugar, and thre

There is a lot of information in this XML: each sentence has a list of Opinions, where each Opinion consists of category, polarity and the target word or phrase. We are really interested only in the raw sentence and the corresponding opinion categories.

There are many ways to parse XML in Python. In this example we use a method that parses XML to Python dict, which is we will then process via loops.

In [ ]:
! pip install xmltodict

In [ ]:
import xmltodict as xd


with open('EN_REST_SB1_TEST.xml.gold','rb') as f:
    d = xd.parse(f)

In [ ]:
d["Reviews"]["Review"][0]["sentences"]["sentence"][0]

OrderedDict([('@id', 'en_BlueRibbonSushi_478218171:0'),
             ('text', 'Yum!'),
             ('Opinions',
              OrderedDict([('Opinion',
                            OrderedDict([('@target', 'NULL'),
                                         ('@category', 'FOOD#QUALITY'),
                                         ('@polarity', 'positive'),
                                         ('@from', '0'),
                                         ('@to', '0')]))]))])

Here is the function that parses the XML and returns a list containing sentences and the corresponding list of categories:

In [ ]:
def read_data(filename):
  result = []
  with open(filename,'rb') as f:
    d = xd.parse(f, force_list=('sentence', 'Opinion'))
  for review in d["Reviews"]["Review"]:
    #print(review)
    for sentence in review["sentences"]["sentence"]:
      
      text = sentence["text"]
      opinion_cats = []
      
      if "Opinions" in sentence and sentence["Opinions"] is not None:
        opinions = sentence["Opinions"]["Opinion"]
        for opinion in opinions:
          opinion_cats.append(opinion["@category"])
      result.append((text, opinion_cats))
  return result



In [ ]:
train_data = read_data("ABSA16_Restaurants_Train_SB1_v2.xml")
test_data = read_data("EN_REST_SB1_TEST.xml.gold")

In [ ]:
print(len(train_data), len(test_data))

2000 676


In [ ]:
train_data[:5]

[('Judging from previous posts this used to be a good place, but not any longer.',
  ['RESTAURANT#GENERAL']),
 ('We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
  ['SERVICE#GENERAL']),
 ('They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
  ['SERVICE#GENERAL']),
 ('The food was lousy - too sweet or too salty and the portions tiny.',
  ['FOOD#QUALITY', 'FOOD#STYLE_OPTIONS']),
 ('After all that, they complained to me about the small tip.',
  ['SERVICE#GENERAL'])]

So, each sentence can have zero or more categories. This task is called multi-label classification, as opposed to single-label classification where each sample corresponds to one and only one category.

The sklearn package has some useful utilities for multi-label classification tasks.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

MultiLabelBinarizer builds a mapping from multi-label labels to IDs, and also constructs a label matrix for our training ans test data.

In [ ]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([set(sample[1]) for sample in train_data])

In [ ]:
train_labels[0:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
mlb.classes_

array(['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY',
       'DRINKS#STYLE_OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY',
       'FOOD#STYLE_OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL',
       'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL'],
      dtype=object)

In [ ]:
test_labels = mlb.transform([set(sample[1]) for sample in test_data])

In [ ]:
test_labels[0:5]

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

So, our labels are now stored as binary matrices -- exactly as we need them.

Let's also construct a list of training inputs, as sklearn likes it:

In [ ]:
train_text = [review[0] for review in train_data]
test_text = [review[0] for review in test_data]

In [ ]:
train_text[0:5]

['Judging from previous posts this used to be a good place, but not any longer.',
 'We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.',
 'They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.',
 'The food was lousy - too sweet or too salty and the portions tiny.',
 'After all that, they complained to me about the small tip.']

Now we can train a simple multi-label classifier.

We use Sklearn's OneVsRestClassifier to do this. This basically builds a seperate base classifier for each of our labels.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier

text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf',  OneVsRestClassifier(LogisticRegression()))
 ])

In [ ]:
text_clf.fit(train_text, train_labels )

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=1.0,
                                                                  class_weight=None,
                      

Classifier is trained. Let's first see how can we apply it on test data. What comes out when we feed it some test data?

In [ ]:
test_predictions = text_clf.predict(test_text)

In [ ]:
test_predictions[0:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
test_labels[0:5]

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

So, the output of our classifier is also a binary matrix. We can evaluate the performance of our model using F1 measure (which is a geometric mean of precision and recall). 

F1 measure can be computed for each label independently and then merged using either macro or micro averaging. We will use micro-averaging that sums up the individual true positives, false positives, and false negatives of the system for different sets and the apply them to get the statistics. 



In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
f1_score(test_labels,
         test_predictions,
         average='micro')

0.4503441494591937

In [ ]:
print(classification_report(test_labels, test_predictions, target_names=mlb.classes_, zero_division=0))

                          precision    recall  f1-score   support

        AMBIENCE#GENERAL       1.00      0.21      0.35        57
           DRINKS#PRICES       0.00      0.00      0.00         3
          DRINKS#QUALITY       0.00      0.00      0.00        21
    DRINKS#STYLE_OPTIONS       0.00      0.00      0.00        12
             FOOD#PRICES       0.00      0.00      0.00        22
            FOOD#QUALITY       0.78      0.54      0.64       226
      FOOD#STYLE_OPTIONS       0.00      0.00      0.00        48
        LOCATION#GENERAL       0.00      0.00      0.00        13
      RESTAURANT#GENERAL       0.84      0.23      0.36       142
RESTAURANT#MISCELLANEOUS       0.00      0.00      0.00        33
       RESTAURANT#PRICES       0.00      0.00      0.00        21
         SERVICE#GENERAL       0.95      0.43      0.60       145

               micro avg       0.84      0.31      0.45       743
               macro avg       0.30      0.12      0.16       743
        

It's now your task to improve this.

You can try a lot of things to make the classifier more accurate. Of course, it's very recommended to try different DNN based approaches. 

Note that there are only 2000 training sentences. This makes the task basically an excercise of transfer learning. You can try using pre-trained word embeddings, pre-trained models like BERT, etc.

Note that the labels have some structure in them. Each label consists of two parts: e.g. FOOD#QUALITY consists of FOOD and QUALITY. Maybe try splitting the labels into two parts and predicting each part independanty? Of course, you need to glue them back up when doing evaluation.

Multi-label classification can be easily handled with neural network models with a model that has one output for each label, but instead of simple cross-entropy, binary cross-entropy has to be used for optimization.

Multi-label classification is a quite popular task and you can find a lot of tutorials on the internet. For example, many many text classification where a text can be assigned many tags (as in StackOverflow) are multi-label tasks.


Check the slides of Lecture 1 on how the final project should look like, what parts it should contain and how it will be graded.


